In [1]:
from tool.functions import *

In [2]:
download_all_bidixes()

2018-06-11 22:02:17,226 | INFO : Start
2018-06-11 22:13:57,475 | INFO : Finish


For now I delete ces-ces, ita-ita, eus-eu_bis, hbs_HR-hbs_SR, zh_CN-zh_TW

In [3]:
%time monodix()

2018-06-11 22:14:29,510 | INFO : started
2018-06-11 22:17:52,316 | INFO : finished
Wall time: 3min 22s


In [ ]:
from tool.functions import get_relevant_languages

In [ ]:
%time get_relevant_languages('spa', 'eng')

In [ ]:
%time load_file('spa', 'eng')

In [ ]:
%time change_encoding('spa-eng')

In [ ]:
%time G = built_from_file('spa-eng')